In [50]:
from Bio import SeqIO, AlignIO
import os, itertools, subprocess
import pandas
import time

In [51]:
genbank = ["BDIP_ARG.gb", "BPUB_RS.gb", "Bothrops_jararaca-NC_030760.1.gb", "Bothrops_pubescens-NC_039648.1.gb", "Bothrops_diporus-NC_039649.1.gb"]
print(len(genbank))

5


In [52]:
fasta_set = set()

def feature_parser(genbank):
    all_features = dict()
    for gb in genbank:
        recovered_features = list()
        print(gb, "\n")
        gb_handle = SeqIO.read(gb, "genbank")
        organism = gb_handle.annotations.get("organism").replace(" ", "_")
        all_features[organism] = {}
        for feature in gb_handle.features:
            gene_name = ''
            seq = feature.location.extract(gb_handle.seq)
            if feature.type == "CDS":
                #continue
                #print(feature.qualifiers.get('gene')[0])
                gene_name = feature.qualifiers.get('gene')[0]
            elif feature.type == "tRNA":
                #continue
                #print(feature.qualifiers.get('product')[0])
                gene_name = feature.qualifiers.get('product')[0]
            elif feature.type == "rRNA":
                if feature.qualifiers.get('product')[0].startswith('small'):
                    feature.qualifiers['product'] = ['12S ribosomal RNA']
                if feature.qualifiers.get('product')[0].startswith('large'):
                    feature.qualifiers['product'] = ['16S ribosomal RNA']
                #print(feature.qualifiers.get('product')[0].split(" ")[0],'\n',seq)
                gene_name = feature.qualifiers.get('product')[0].split(" ")[0]
            if gene_name != '':
                if gene_name in recovered_features:
                    gene_name = "{}2".format(gene_name)
                recovered_features.append(gene_name)
                feature_dict = {gene_name : seq}
                all_features[organism].update(feature_dict)
                
                #with open(fasta_name, "a+") as fasta:
                    #fasta_set.add(fasta_name)
                    #fasta.write(">{}_{}\n{}\n".format(organism,gene_name,seq))
                #print(header)
                #print(recovered_features)
                #print(len(fasta_set))
                #print(fasta_set)
                #print('>{} {}'.format(header, organism))
        #print("----------------------------\n")
    return(all_features)

feature_dict = feature_parser(genbank)

BDIP_ARG.gb 



FileNotFoundError: [Errno 2] No such file or directory: 'BDIP_ARG.gb'

In [ ]:
feature_dict

In [ ]:
for gene in feature_dict.values():
        print(gene)

In [ ]:
def remove_feature_from_dict(feature):
    for gene in feature_dict.values():
        if feature in gene.keys():
            del gene[feature]
        
def count_features(feature_dict):
    counter = dict()
    for gene in feature_dict.values():
        for i in gene.keys():
            if i not in counter:
                counter[i] = 1
            else:
                counter[i] += 1
    return(counter)

def remove_missing_features(feature_dict):                
    counter = count_features(feature_dict)
    print(counter)
    for key, value in counter.items():
        if value < len(feature_dict):
            print(key)
            remove_feature_from_dict(key)
    return(feature_dict)

feature_dict = remove_missing_features(feature_dict)

    
#new_feature_dict = remove_missing_features(feature_dict)
        
#print(len(feature_dict))
#print(count_features)
    

In [ ]:
for i in feature_dict.keys():
    print(feature_dict[i].keys(), "\n")

In [ ]:
def create_folder(sp1, sp2):
    folder_name = "{}_vs_{}".format(sp1, sp2)
    os.mkdir(folder_name)
    os.chdir(folder_name)
    print("Running clustal alingment for {}".format(folder_name))

def run_clustalo(sp1, sp2, feature_dict):
    sp1_feat = feature_dict[sp1]
    sp2_feat = feature_dict[sp2]
    clustal_list = list()
    for i in sp1_feat.keys():
        clustal_output = "{}.fa".format(i)
        clustal_error = "{}.err".format(i)
        with open("tempfile", "w+") as tempfile, open(clustal_output, "w+") as clustal_out, open(clustal_error, "w+") as clustal_err:
            tempfile.write(">{}\n{}\n>{}\n{}".format(sp1, sp1_feat[i], sp2, sp2_feat[i]))
            tempfile.close()
            #time.sleep(2)
            alignment = subprocess.Popen(["clustalo", "-i", "tempfile"], stdout=clustal_out, stderr=clustal_err)
            alignment.wait()
            os.remove("tempfile")
        if os.path.getsize(clustal_output) != 0 and os.path.getsize(clustal_error) == 0:
            os.remove(clustal_error)
            clustal_list.append(clustal_output)
        else:
            print("There was a problem with the {}_vs{} alignment. Check the file {}".format(sp1, sp2, clustal_error))
    return(clustal_list)

def ambiguous_sites_as_gaps(clustal_list):
    print("Replacing ambiguous sites with gap characters...")
    data = str()
    for aln in clustal_list:
        data = str()
        data2 = str()
        print(aln)
        alignment = AlignIO.read(aln, "fasta")
        for index in [0, 1]: ##Index of each fasta record in the alignment - in this case, 0 and 1 (pairwise)
            valid_nucs = ["A", "T", "G", "C"]
            unambiguous_seq = str()
            for nuc in alignment[index].seq:
                if nuc in valid_nucs:
                    unambiguous_seq += nuc
                else:
                    unambiguous_seq += "-"
            data += ">{} : {}\n".format(alignment[index].id, len(unambiguous_seq))
            data2 += ">{}\n{}\n".format(alignment[index].id, unambiguous_seq)
        print(data)
        with open(aln, "w") as unambiguous_aln:
            unambiguous_aln.write(data2)

def concat_alignment(clustal_list):
    ##Besides the merging, this function needs to generate a clustal list with all the absolute paths to
    ##the final alignment files, to be used as input to the "alignment_parser function"
    header1 = set()
    header2 = set()
    seq1 = ""
    seq2 = ""
    size = 0
    number_of_features = 0
    concat_filename = ""
    for num, i in enumerate(clustal_list, 1):
        aln = AlignIO.read(i, "fasta")
        header1.add(aln[0].id)
        header2.add(aln[1].id)
        seq1 += aln[0].seq
        seq2 += aln[1].seq
        size += aln.get_alignment_length()
        number_of_features = num
    if len(header1) == len(header2) == 1 and size == len(seq1) == len(seq2):
        header1 = list(header1)[0]
        header2 = list(header2)[0]
        concat_filename = "{}_vs_{}_concat.fa".format(header1, header2)
        print(header1, header2)
        print(len(seq1), len(seq2))
        print("Features: {}, Alignment_size: {}".format(number_of_features, size))
        with open(concat_filename, "w") as concat:
            concat.write(">{}\n{}\n>{}\n{}\n".format(header1, seq1, header2, seq2))
    else:
        print("There was a problem in the merging of alignments in {}".format(concat_filename))
        return(None)
    return(os.path.abspath(concat_filename))
    
def pairwise_permutations(feature_dict):
    '''Receives a dictionary of multiple sequence entries (e.g. {header1 : seq1, header2 : seq2, ... headerN : seqN}) and returns a list of all possible pairwise permutations between the dictionary's keys.'''
    permutations = list()
    global concat_list
    concat_list = list()
    for sp1, sp2 in itertools.combinations(feature_dict.keys(), 2):
        create_folder(sp1, sp2)
        clustal_list = run_clustalo(sp1, sp2, feature_dict)
        ambiguous_sites_as_gaps(clustal_list)
        concat_list.append(concat_alignment(clustal_list))
        os.chdir("..")        
        
pairwise_permutations(feature_dict)

In [ ]:
def ambiguous_sites_as_gaps(clustal_list):
    print("Replacing ambiguous sites with gap characters...")
    data = str()
    for aln in clustal_list:
        data = str()
        data2 = str()
        print(aln)
        alignment = AlignIO.read(aln, "fasta")
        for index in [0, 1]: ##Index of each fasta record in the alignment - in this case, 0 and 1 (pairwi)
            valid_nucs = ["A", "T", "G", "C"]
            unambiguous_seq = str()
            for nuc in alignment[index].seq:
                if nuc in valid_nucs:
                    unambiguous_seq += nuc
                else:
                    unambiguous_seq += "-"
            data += ">{} : {}\n".format(alignment[index].id, len(unambiguous_seq))
            data2 += ">{}\n{}\n".format(alignment[index].id, unambiguous_seq)
        print(data)
        with open(aln, "w") as unambiguous_aln:
            unambiguous_aln.write(data2)

In [ ]:
def unambiguous_check_length(clustal_list):
    for file in clustal_list:
        length = set()
        aln = AlignIO.read(file, "fasta")
        for i in range(len(aln)):
            length.update([len(aln[i])])
        if len(length) == 1:
            print("The sequences in {} have the same length - {} - Conversion to unambiguous SUCCESSFUL!".format(file, length))
        else:
            print("The sequences in {} have different lengths - {} - Conversion to unambiguous FAILED!".format(file, length))
            

In [ ]:
def clustal_parser(clustal_list):
    fields = ["Query MT", "Subject MT", "Total Alignment Sites", "Gapped Sites", "Valid Sites", "Identical Sites", "Variant Sites", "Identity Percentage"]
    excel_output_dict = {k : list() for k in fields}
    for file in clustal_list: ##First, we need to get each information from the alignment and assign it to a variable
        aln = AlignIO.read(file, "fasta")
        query_seq = aln[0].id
        subject_seq = aln[1].id
        total_aln_sites = aln.get_alignment_length()
        gap_sites = list()
        gap_sites_count = 0
        identical_sites = list()
        identical_sites_count = 0
        variant_sites = list()
        variant_sites_count = 0
        collumns = dict()
        for i in range(1, total_aln_sites + 1):
            collumns.update({i : aln[:, i-1]})
        for k, v in collumns.items(): ##Get number of gapped sites
            if "-" in v:
                gap_sites_count += 1
                gap_sites.append([k, v])
                #print(k, v)
                continue
            nucs_in_collumn = set()
            for i in [0,1]: ##Get number of identical and variable sites for pairwise alignment
                nucs_in_collumn.update([v[i]])
            if len(nucs_in_collumn) == 1:
                identical_sites_count += 1
                identical_sites.append([k, v])
            if len(nucs_in_collumn) > 1:
                variant_sites_count += 1
                variant_sites.append([k, v])
                #print(k, v)        
        valid_sites = total_aln_sites - gap_sites_count
        identity_percentage = round((identical_sites_count / valid_sites) * 100, 2)
        
        
        ##After getting the variables, the script generates a file for each pairwise alignment, with a summary of the alignment's data
        
        parser_output = "{}.clustalparser".format(file[:-3])        
        with open(parser_output, "w+") as cluparser:
            cluparser.write("Query sequence: {} - A {} \t T {} \t G {} \t C {} \t gaps {}\n".format(query_seq, aln[0].seq.count("A"), aln[0].seq.count("T"), aln[0].seq.count("G"), aln[0].seq.count("C"), aln[0].seq.count("-")))
            cluparser.write("Subject sequence: {} - A {} \t T {} \t G {} \t C {} \t gaps {}\n".format(subject_seq, aln[1].seq.count("A"), aln[1].seq.count("T"), aln[1].seq.count("G"), aln[1].seq.count("C"), aln[1].seq.count("-")))
            cluparser.write("Total alignment size: {}\n".format(total_aln_sites))
            cluparser.write("Gapped columns: {}\n".format(gap_sites_count))
            cluparser.write("Valid columns (Total - Gapped): {}\n".format(valid_sites))
            cluparser.write("Identical columns: {}\n".format(identical_sites_count))
            cluparser.write("Variant columns: {}\n".format(variant_sites_count))
            cluparser.write("Identity percentage - ((Identical collumns / Valid collumns) * 100): {} %\n".format(identity_percentage))
            cluparser.write("VARIANT SITES:\n")
            cluparser.write("Number\tAlignment_site\t{}_nt\t{}_nt\n".format(query_seq, subject_seq))
            [cluparser.write("{}\t{}\t{}\t{}\n".format(i, v[0], v[1][0], v[1][1])) for i, v in enumerate(variant_sites, 1)]
            cluparser.write("GAP SITES:\n")
            cluparser.write("Number\tAlignment_site\t{}_nt\t{}_nt\n".format(query_seq, subject_seq))
            [cluparser.write("{}\t{}\t{}\t{}\n".format(i, v[0], v[1][0], v[1][1])) for i, v in enumerate(gap_sites, 1)]
            cluparser.write("IDENTICAL SITES:\n")
            cluparser.write("Number\tAlignment_site\t{}_nt\t{}_nt\n".format(query_seq, subject_seq))
            [cluparser.write("{}\t{}\t{}\t{}\n".format(i, v[0], v[1][0], v[1][1])) for i, v in enumerate(identical_sites, 1)]

        ##Lastly, the script will save all data to a dictionary and return it. This dict will be used to create a pandas dataframe and write a excel spreadsheet
            
        excel_output_dict.get("Query MT").append(query_seq)
        excel_output_dict.get("Subject MT").append(subject_seq)
        excel_output_dict.get("Total Alignment Sites").append(total_aln_sites)
        excel_output_dict.get("Gapped Sites").append(gap_sites_count)
        excel_output_dict.get("Valid Sites").append(valid_sites)
        excel_output_dict.get("Identical Sites").append(identical_sites_count)
        excel_output_dict.get("Variant Sites").append(variant_sites_count)
        excel_output_dict.get("Identity Percentage").append(identity_percentage)
            
        #if (gap_sites_count + identical_sites_count + variant_sites_count) == total_aln_sites:
         #   print("The sum is correct")
        #else:
        #    print("There was a problem with the parsing")
        #print(aln[1].seq.count("-"))
        
    return(excel_output_dict)

In [ ]:
def generate_excel_aln_summary(excel_output_dict):
    aln_summary = pandas.DataFrame(excel_output_dict)
    aln_summary.to_excel("Pairwise_alignment_summary.xlsx", index=False, sheet_name="Aln_summary")    

In [ ]:
excel = clustal_parser(concat_list)
generate_excel_aln_summary(excel)

# What to do from here:

- [x] Create the pairwise comparison list using the function from the other script
- [x] For each pairwise comparison, create a folder
- [x] Inside each folder, there will be an fasta for each gene pair
- [x] Each fasta will be used for an alignment
- [ ] Lastly, the sequece of all alignments will be merged into a single supermatrix (could be in any order, maybe the synteny order?)
- [ ] This supermatrix will be used as input for the clustal_parser function